In [17]:
%pip install os
%pip install pandas
%pip install numpy  
%pip install sklearn 
%pip install statsmodels 

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by o

In [9]:
import os 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm

In [14]:
path = './extr.xlsx'
# 读取数据
df = pd.read_excel(rf'{path}')

# 提取所需列
columns = ['母亲年龄', '婚姻状况', '教育程度', '妊娠时间（周数）', 'CBTS', 'EPDS', 'HADS', '婴儿行为特征', '整晚睡眠时间（时：分：秒）', '睡醒次数','入睡方式']
data = df[columns][:390]

# 清洗时间数据并计算实际入睡时间和平均入睡时间
data['整晚睡眠时间（时：分：秒）'] = pd.to_datetime(data['整晚睡眠时间（时：分：秒）'], format='%H:%M:%S', errors='coerce')
data = data.dropna(subset=['整晚睡眠时间（时：分：秒）'])
data['实际入睡时间'] = data['整晚睡眠时间（时：分：秒）'].dt.hour + data['整晚睡眠时间（时：分：秒）'].dt.minute / 60 + data['整晚睡眠时间（时：分：秒）'].dt.second / 3600
average_bedtime = data['实际入睡时间'].mean()

# 计算睡眠效率并转换婴儿行为特征列数据类型
data['标准入睡时间'] = average_bedtime
data['睡眠效率'] = (data['实际入睡时间'] / data['标准入睡时间']) * 100
data['婴儿行为特征'] = data['婴儿行为特征'].map({'安静型': 0, '中等型': 1, '矛盾型': 2}).astype(float)

# 提取自变量和因变量
x_columns = ['母亲年龄', '婚姻状况', '教育程度', '妊娠时间（周数）', 'CBTS', 'EPDS', 'HADS']
x_data = data[x_columns]
y_baby_behavior = data['婴儿行为特征']
y_sleep_efficiency = data['睡眠效率']

# 创建模型和变量字典
models = {
    '婴儿行为特征': {},
    '睡眠效率': {}
}

# 回归模型拟合
for column in x_columns:
    x = sm.add_constant(x_data[column])
    model_baby_behavior = sm.OLS(y_baby_behavior, x).fit()
    model_sleep_efficiency = sm.OLS(y_sleep_efficiency, x).fit()
    models['婴儿行为特征'][column] = model_baby_behavior
    models['睡眠效率'][column] = model_sleep_efficiency

# 打印模型拟合结果
for target in models:
    print(f"{target}模型拟合结果：")
    for column in models[target]:
        print(models[target][column].summary())

婴儿行为特征模型拟合结果：
                            OLS Regression Results                            
Dep. Variable:                 婴儿行为特征   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     5.122
Date:                Tue, 22 Aug 2023   Prob (F-statistic):             0.0242
Time:                        22:10:48   Log-Likelihood:                -364.76
No. Observations:                 389   AIC:                             733.5
Df Residuals:                     387   BIC:                             741.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2962      0.218      

In [15]:

# 训练模型
def train_model():
    # 读取数据
    df = pd.read_excel(rf"{path}")
    df=df[:-20]

    # 提取特征和目标变量
    x_columns = ['母亲年龄', '婚姻状况', '教育程度',  'CBTS', 'EPDS', 'HADS']
    y_column = '婴儿行为特征'

    x_data = df[x_columns]
    y_data = df[y_column]

    # 创建随机森林分类器
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

    # 训练模型
    rf_classifier.fit(x_data, y_data)

    return rf_classifier

# 预测缺失数据
def predict_missing_data(model):
    # 读取倒数20行的数据
    df = pd.read_excel(rf"{path}")
    df_missing = df.tail(20)

    x_missing = df_missing[['母亲年龄', '婚姻状况', '教育程度',  'CBTS', 'EPDS', 'HADS']]

    # 进行预测
    y_pred = model.predict(x_missing)

    # 输出预测结果
    print("预测结果：")
    print(y_pred)

# 训练模型
model = train_model()

# 预测缺失数据
predict_missing_data(model)

with open(f'{path}','w') as out:
    pass

预测结果：
['中等型' '中等型' '中等型' '中等型' '中等型' '矛盾型' '安静型' '中等型' '中等型' '中等型' '中等型' '中等型'
 '中等型' '安静型' '中等型' '安静型' '中等型' '中等型' '中等型' '中等型']
